# Aquisition RMN
## Functions
Bloc to Run everytime :


In [2]:
%matplotlib qt
#to open matplotlib window

import csv
import matplotlib.pyplot as plt
import numpy as np
import datetime
import paramiko
import os
import time

SFTP_PORT = 22
SFTP_USERNAME = "root"
SFTP_PASSWORD = "root"
REMOTE_PATH = "Pitaya-Tests/" 
REMOTE_FOLDER = "Pitaya-Tests"
SAMPLING_RATE = 125e+6

def run_acquisition_command(hostname, samplesNb, dec, 
                            FidNb, FileName, larmorFrequency, excitationDuration): #voir si file path marche
    port = 22
    username = SFTP_USERNAME
    password = SFTP_PASSWORD
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    filePath = "mesures/" + FileName

    try:
        client.connect(hostname=hostname, username=username, password=password, port=port)          

        print(f"[INFO] Connexion établie à {hostname}")
        command = f"cd {REMOTE_FOLDER} && ./Acquisition_axi.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {excitationDuration} &"
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        errors = stderr.read().decode()

    except Exception as e:
        print(f"[ERREUR] SSH : {e}")

    finally:
        

        client.close()
        print(f"[CMD-SSH] {command}")
        if output:
            print("[OUTPUT SSH]\n", output)
        if errors:
            print("[ERROR SHH]\n", errors)
        print("[INFO] Connexion SSH fermée.")

def run_find_excitation_time_command(hostname, samplesNb, dec, 
                            FidNb, FileName, larmorFrequency, start_excitationDuration, step, wait_time):
    port = 22
    username = SFTP_USERNAME
    password = SFTP_PASSWORD
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    filePath = "balayage/" + FileName

    try:
        client.connect(hostname=hostname, username=username, password=password, port=port)          

        print(f"[INFO] Connexion établie à {hostname}")
        command = f"cd {REMOTE_FOLDER} && ./Find_excitation_time.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {start_excitationDuration} {step} {wait_time} &"
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        errors = stderr.read().decode()

    except Exception as e:
        print(f"[ERREUR] SSH : {e}")

    finally:
        

        client.close()
        print(f"[CMD-SSH] {command}")
        if output:
            print("[OUTPUT SSH]\n", output)
        if errors:
            print("[ERROR SHH]\n", errors)
        print("[INFO] Connexion SSH fermée.")

def run_find_frequency_command(hostname, samplesNb, dec, 
                            FidNb, FileName, larmorFrequency, start_excitationDuration, step, wait_time):
    port = 22
    username = SFTP_USERNAME
    password = SFTP_PASSWORD
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    filePath = "balayage/" + FileName

    try:
        client.connect(hostname=hostname, username=username, password=password, port=port)          

        print(f"[INFO] Connexion établie à {hostname}")
        command = f"cd {REMOTE_FOLDER} && ./Find_Frequency.exe {samplesNb} {dec} {FidNb} {filePath} {larmorFrequency} {start_excitationDuration} {step} {wait_time} &"
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        errors = stderr.read().decode()

    except Exception as e:
        print(f"[ERREUR] SSH : {e}")

    finally:
        

        client.close()
        print(f"[CMD-SSH] {command}")
        if output:
            print("[OUTPUT SSH]\n", output)
        if errors:
            print("[ERROR SHH]\n", errors)
        print("[INFO] Connexion SSH fermée.")

def telecharger_fichier_sftp(nom_Fichier,folder_name, SFTP_HOST):
    """Télécharge le fichier CSV via SFTP"""
    
    try:
        # Connexion SFTP
        transport = paramiko.Transport((SFTP_HOST, SFTP_PORT))
        transport.connect(username=SFTP_USERNAME, password=SFTP_PASSWORD)
        sftp = paramiko.SFTPClient.from_transport(transport)
        
        print(f"Connexion SFTP établie avec {SFTP_HOST}")

        # Créer le dossier local avec timestamp
        now = datetime.datetime.now()
        nom_dossier_local = f"{folder_name}-{now.strftime('%Y%m%d_%H%M%S')}"
        os.makedirs(nom_dossier_local, exist_ok=True)
        
        # Liste des fichiers à télécharger
        fichier_distant = nom_Fichier
        
        # Téléchargement du fichier
        chemin_distant = REMOTE_PATH + folder_name+'/' + fichier_distant
        chemin_local = os.path.join(nom_dossier_local, fichier_distant)
        
        try:
            sftp.get(chemin_distant, chemin_local)
            fichier_local = chemin_local
            print(f"Téléchargé: {fichier_distant}")
        except FileNotFoundError:
            print(f"Fichier non trouvé: {chemin_distant}")
        except Exception as e:
            print(f"Erreur lors du téléchargement de {fichier_distant}: {e}")
    
        # Fermer la connexion
        sftp.close()
        transport.close()
        print("Connexion SFTP fermée")
        
        return fichier_local
        
    except Exception as e:
        print(f"Erreur de connexion SFTP: {e}")
        return ""


def open_file(pathFile_csv):
        voltage_acc = []

        with open(pathFile_csv, 'r', encoding='utf-8') as fichier_:
            lecteur = csv.reader(fichier_)

            # Lire et parser l'entête du fichier
            ligne_entete = next(lecteur)
            dsize = int(ligne_entete[0])
            decimation = int(ligne_entete[1])
            nombre_de_FID = int(ligne_entete[2])
            gain = float(ligne_entete[3])
            offset = float(ligne_entete[4])
            nb_bits = int(ligne_entete[5])

            # Initialisation
            voltage = [[] for _ in range(nombre_de_FID)]
            mean = []
            voltage_acc = np.zeros(dsize)  # Initialiser au bon format et taille

            # Lecture et conversion des tensions
            for j in range(nombre_de_FID):
                ligne = next(lecteur)

                signal = []

                # for val_bin in ligne:
                #     #val_bin = int(val_bin)
                #     val = convert_to_volt(val_bin, nb_bits, gain, offset)
                #     signal.append(val)  # pas besoin de np.append ici


                for val in ligne:
                    signal.append(float(val))

                # Convertir la ligne en tableau numpy
                signal = np.array(signal)

                #### Calcul de la moyenne et centrage
                #moyenne = np.mean(signal)
                #signal_centre = signal - moyenne

                # Stocker les données
                voltage[j] = signal
                #mean.append(moyenne)

                # Accumulation du signal total
                voltage_acc += signal

        # Création du tableau temps
        duree_mesure = (dsize * decimation) / SAMPLING_RATE 
        time = np.linspace(0, duree_mesure, dsize, endpoint=False)

        # Calcul de la moyenn et centrage du signal accumulé
        moyenne = np.mean(voltage_acc[-100000:-1000])
        voltage_acc = voltage_acc - moyenne

        print(f"Fichier {pathFile_csv} lu. {len(voltage)} signaux FID chargés.")
        return time, voltage, voltage_acc
    

def plot_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    
    amountFID = len(voltage_matrix)

    plt.figure(figsize=(12, 7))

    # Affichage des courbes FID
    for w in range(amountFID):
        plt.plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')

    # Affichage du signal accumulé
    plt.plot(time_axis, voltage_accumulated_axis, marker='+', linestyle='-', label='Total', linewidth=2, color='black')

    # Mise en forme du graphique
    plt.title(f'Superposition - {graph_name}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Tension (V)')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

def subpolts_acc(graph_name, time_axis, voltage_matrix, voltage_accumulated_axis):
    # Supposons que voltage_matrix et time_axis soient déjà définis
    amountFID = len(voltage_matrix)

    # Créer une figure et un ensemble de sous-graphes
    fig, axs = plt.subplots(amountFID, 1, figsize=(12, 7 * amountFID), sharex=True)

    # Affichage des courbes FID sur des sous-graphes séparés
    for w in range(amountFID):
        axs[w].plot(time_axis, voltage_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')
        axs[w].set_title(f'FID {w+1}')
        axs[w].set_ylabel('Tension (V)')
        axs[w].grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        axs[w].legend(loc='upper right')

    # Mise en forme globale du graphique
    fig.suptitle(f'Superposition - {graph_name}', y=1.02)  # y est légèrement ajusté pour éviter le chevauchement
    plt.xlabel('Temps (s)')
    plt.tight_layout()
    plt.show()


def plot_fourier_transform(time, voltage):
    # Ensure time and voltage are numpy arrays
    time = np.array(time)
    voltage = np.array(voltage)

    # Sampling interval and frequency
    dt = time[1] - time[0]
    fs = 1 / dt

    # Compute FFT
    N = len(voltage)
    fft_values = np.fft.fft(voltage)
    freq = np.fft.fftfreq(N, dt)

    # Keep only the positive frequencies
    #idx = np.where(freq >= 0)
    #freq = freq[idx]
    magnitude = np.abs(fft_values) * 2 / N  # Normalize amplitude

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(freq, magnitude)
    plt.title("Fourier Transform")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## Run program
Open the Redpitaya webpage before running : [rp-f0d66d.local](http://rp-f0d66d.local/)

In [ ]:
hostName = "rp-f09fd8.local"
#===== PARAMETERS ======

sample_Amount = 524288/2               #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 8                      #Decimation
aquisiton_Amount = 1               #Nubmer of acquisitons
experiment_name = "test.csv"            #name of the file of the measured tension
larmor_Frequency_Hertz = 12348104.151          #Larmor frequency
excitation_duration_seconds = 50.022e-6     #Excitation time


total_time = (sample_Amount * decimation)/SAMPLING_RATE
print(f"temps mesuré : {total_time}s")
print(f"temps total : {total_time*aquisiton_Amount}")

nb_cycles = larmor_Frequency_Hertz*excitation_duration_seconds
print(f"nb cycles burst : {nb_cycles}")


temps mesuré : 0.016777216s
temps total : 0.016777216
nb cycles burst : 1217.9408658413222


In [ ]:
run_acquisition_command(hostName,
                        sample_Amount, 
                        decimation, 
                        aquisiton_Amount, 
                        experiment_name, 
                        larmor_Frequency_Hertz, 
                        excitation_duration_seconds)


In [88]:
pathFile_csv = telecharger_fichier_sftp(experiment_name,hostName)
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)
plot_acc(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)


Connexion SFTP établie avec rp-f09fd8.local
Téléchargé: test.csv
Connexion SFTP fermée
Fichier mesures-20250827_120817\test.csv lu. 10 signaux FID chargés.


In [31]:
plot_fourier_transform(time_array, voltageAcc_array)

In [ ]:
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)
plot_acc(experiment_name, time_array, voltage_array_matrix, voltageAcc_array)
plot_fourier_transform(time_array, voltageAcc_array)

Fichier mesures-20250821_141425\test_water_puredevice_tf_lowdecimation.csv lu. 10 signaux FID chargés.


## Plot previous acquisiton

In [40]:

nomDossier = "mesures-20250821_143040"
nomFichier_local = "test_water_puredevice_tf_lowdecimation.csv"

pathFile_csv_new = nomDossier +'/'+ nomFichier_local
#pathFile_csv_new = pathFile_csv

time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv_new)
plot_acc(nomFichier_local, time_array[3000:-1000], voltage_array_matrix[:][3000:-1000], voltageAcc_array[3000:-1000])
plot_fourier_transform(time_array[3000:-1000], voltageAcc_array[3000:-1000])

Fichier mesures-20250821_143040/test_water_puredevice_tf_lowdecimation.csv lu. 10 signaux FID chargés.


___

# Balayage temps d'excitation et frequence
Balayage Excitation :

In [8]:
hostName = "rp-f0d66d.local"
#===== PARAMETERS ======

sample_Amount = 1216             #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 1024                 #Decimation
experiment_name = "balayage_time.csv"            #name of the file of the measured tension
folder_name = "balayage"
larmor_Frequency_Hertz = 12348104.151          #Larmor frequency fixed while the excitation time sweep
Start_excitation_duration_seconds = 100.022e-6     #Excitation time start
Nb_of_step = 2               #Nubmer of acquisitons
Step_excitation_time_seconds = 100e-6
Wait_time_seconds = 1

run_find_excitation_time_command(hostName, sample_Amount, decimation, 
                            Nb_of_step, experiment_name, larmor_Frequency_Hertz, 
                            Start_excitation_duration_seconds, Step_excitation_time_seconds, Wait_time_seconds)

pathFile_csv = telecharger_fichier_sftp(experiment_name,folder_name,hostName)
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)

amountFID = len(voltage_array_matrix)

plt.figure(figsize=(12, 7))

# Affichage des courbes FID
for w in range(amountFID):
    plt.plot(time_array, voltage_array_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')


# Mise en forme du graphique
plt.title(f'Superposition - {folder_name}/{experiment_name}')
plt.xlabel('Temps (s)')
plt.ylabel('Tension (V)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()


[INFO] Connexion établie à rp-f0d66d.local
[CMD-SSH] cd Pitaya-Tests && ./Find_excitation_time.exe 1216 1024 2 balayage/balayage_time.csv 12348104.151 0.000100022 0.0001 1 &
[OUTPUT SSH]
 larmor 12348104.000000, duration excitation 0.000100
fichier crée : balayage/balayage_time.csv
Balayage de 100us a 200us, pas de 10us
temps d'excitation 0.000200
nombre des periodes dans le burst 2469
wait to be filled
Tr pos1: 0x1
ecriture FID 0
Balayage de 100us a 200us, pas de 10us
temps d'excitation 0.000300
nombre des periodes dans le burst 3704
wait to be filled
Tr pos1: 0x0
ecriture FID 1
Temps d'execution : 0.026512

[INFO] Connexion SSH fermée.
Connexion SFTP établie avec rp-f0d66d.local
Téléchargé: balayage_time.csv
Connexion SFTP fermée
Fichier balayage-20250827_155125\balayage_time.csv lu. 2 signaux FID chargés.


Balayage fréquence :

In [10]:
hostName = "rp-f0d66d.local"
#===== PARAMETERS ======

sample_Amount = 1216             #Number of points measured (MAX = 524288) must be a multiple of 2
decimation = 1024                 #Decimation
experiment_name = "balayage_freq.csv"            #name of the file of the measured tension
folder_name = "balayage"
excitation_duration_seconds = 50.022e-3     #Excitation time start
start_larmor_Frequency_Hertz = 1000          #Larmor frequency fixed while the excitation time sweep
Nb_of_step = 2               #Nubmer of acquisitons
Step_frequency_hertz = 1e+3
Wait_time_seconds = 0

run_find_frequency_command(hostName, sample_Amount, decimation, 
                            Nb_of_step, experiment_name, start_larmor_Frequency_Hertz, 
                            excitation_duration_seconds, Step_frequency_hertz, Wait_time_seconds)

pathFile_csv = telecharger_fichier_sftp(experiment_name,folder_name,hostName)
time_array, voltage_array_matrix, voltageAcc_array = open_file(pathFile_csv)

amountFID = len(voltage_array_matrix)

plt.figure(figsize=(12, 7))

# Affichage des courbes FID
for w in range(amountFID):
    plt.plot(time_array, voltage_array_matrix[w], marker='+', linestyle='-', label=f'FID {w+1}')


# Mise en forme du graphique
plt.title(f'Superposition - {folder_name}/{experiment_name}')
plt.xlabel('Temps (s)')
plt.ylabel('Tension (V)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()


[INFO] Connexion établie à rp-f0d66d.local
[CMD-SSH] cd Pitaya-Tests && ./Find_Frequency.exe 1216 1024 2 balayage/balayage_freq.csv 1000 0.050022 1000.0 0 &
[OUTPUT SSH]
 larmor 1000.000000, duration excitation 0.050022
fichier crée : balayage/balayage_freq.csv
Balayage de 100us a 200us, pas de 10us
frequence de larmor 2000.000000
nombre des periodes dans le burst 100
wait to be filled
Tr pos1: 0x0
ecriture FID 0
Balayage de 100us a 200us, pas de 10us
frequence de larmor 3000.000000
nombre des periodes dans le burst 150
wait to be filled
Tr pos1: 0x0
ecriture FID 1
Temps d'execution : 0.069069

[INFO] Connexion SSH fermée.
Connexion SFTP établie avec rp-f0d66d.local
Téléchargé: balayage_freq.csv
Connexion SFTP fermée
Fichier balayage-20250827_155557\balayage_freq.csv lu. 2 signaux FID chargés.
